In [1]:
import os

os.chdir("/Users/ibulmnie/Documents/20241/BigData/crypto-big-data/")

In [1]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType
from pyspark.sql.functions import col, from_json, to_date, avg, lit, to_timestamp
from pyspark.sql import functions as F
from pyspark.sql.window import Window
import time
import shutil
from datetime import datetime, timedelta
import os
from pyspark.sql import functions as F

gcs_jar_path = os.path.abspath("config/gcs-connector-hadoop3-latest.jar")

# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("hehee") \
    .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")\
    .config("spark.hadoop.fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")\
    .config("spark.hadoop.fs.gs.auth.service.account.enable", "true")\
    .config("spark.hadoop.fs.gs.auth.service.account.json.keyfile", "config/key/btcanalysishust-495a3a227f22.json") \
    .config("spark.jars", gcs_jar_path) \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0,io.delta:delta-core_2.12:2.4.0")\
    .config("spark.hadoop.fs.gs.project.id", "btcanalysishust")\
    .getOrCreate()


24/12/14 01:49:26 WARN Utils: Your hostname, MacBook-Pro-cua-My.local resolves to a loopback address: 127.0.0.1; using 192.168.1.6 instead (on interface en0)
24/12/14 01:49:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/ibulmnie/Documents/20241/BigData/crypto-big-data/spark-env/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/ibulmnie/.ivy2/cache
The jars for the packages stored in: /Users/ibulmnie/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c45ff2a3-b2f1-4f5c-a727-6a1a7669294a;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.0 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
	found io

24/12/14 01:49:45 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [3]:
from pyspark.sql.types import StructType, StructField, TimestampType, DoubleType


In [4]:
# Schema của dữ liệu Kafka
schema = StructType([
    StructField("timestamp", StringType()),
    StructField("prices", StructType([
        StructField("bitcoin", FloatType()),
        StructField("ethereum", FloatType()),
        StructField("tether", FloatType()),
        StructField("usd-coin", FloatType()),
        StructField("ripple", FloatType()),
        StructField("cardano", FloatType()),
        StructField("dogecoin", FloatType()),
        StructField("matic-network", FloatType()),
        StructField("solana", FloatType()),
        StructField("litecoin", FloatType()),
        StructField("polkadot", FloatType()),
        StructField("shiba-inu", FloatType()),
        StructField("tron", FloatType()),
        StructField("cosmos", FloatType()),
        StructField("chainlink", FloatType()),
        StructField("stellar", FloatType()),
        StructField("near", FloatType()),
    ]))
])

# Đọc dữ liệu từ Kafka
kafka_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "35.206.252.44:9092") \
    .option("subscribe", "crypto-pricessss") \
    .option("startingOffsets", "latest") \
    .option("maxOffsetsPerTrigger", 1000)\
    .load()


parsed_df = kafka_df.selectExpr("CAST(value AS STRING)") \
    .select(F.from_json(F.col("value"), schema).alias("data"))

crypto_parsed_df = parsed_df.select(
    F.to_timestamp(F.col("data.timestamp"), "yyyy-MM-dd'T'HH:mm:ss.SSSSSSXXX").alias("DATE"),
    F.col("data.prices.bitcoin").alias("CLOSE")
)

    # F.col("data.prices.ethereum").alias("ETH"),
    # F.col("data.prices.tether").alias("USDT"),
    # F.col("data.prices.usd-coin").alias("USDC"),
    # F.col("data.prices.ripple").alias("XRP"),
    # F.col("data.prices.cardano").alias("ADA"),
    # F.col("data.prices.dogecoin").alias("DOGE"),
    # F.col("data.prices.matic-network").alias("MATIC"),
    # F.col("data.prices.solana").alias("SOL"),
    # F.col("data.prices.litecoin").alias("LTC"),
    # F.col("data.prices.polkadot").alias("DOT"),
    # F.col("data.prices.shiba-inu").alias("SHIB"),
    # F.col("data.prices.tron").alias("TRX"),
    # F.col("data.prices.cosmos").alias("ATOM"),
    # F.col("data.prices.chainlink").alias("LINK"),
    # F.col("data.prices.stellar").alias("XLM"),
    # F.col("data.prices.near").alias("NEAR")
# )

# # Định nghĩa schema cho dữ liệu lịch sử
historical_schema = StructType([
    StructField("BASE", StringType(), True),#là mã của crypto, ví dụ BASE BTC sẽ có các row chứa dữ liệu theo các cột kế bên
    StructField("DATE", TimestampType(), True),
    StructField("OPEN", FloatType(), True),
    StructField("HIGH", FloatType(), True),
    StructField("LOW", FloatType(), True),
    StructField("CLOSE", FloatType(), True),
    StructField("VOLUME", FloatType(), True),
    StructField("YEAR", StringType(), True),
    StructField("MONTH", StringType(), True),
    StructField("__index_level_0__", StringType(), True)
])


In [ ]:
# 1. Đọc dữ liệu batch từ Parquet
historical_data_df = (
    spark.read.format("parquet")
    .load("gs://crypto-historical-data-2/ver2/BTC/2024/*")
    .select(F.col("DATE").cast("timestamp"), "CLOSE")
)

# 3. Trigger micro-batch và xử lý
def process_batch(micro_batch_df, batch_id):
    # Lọc ra giá trị gần nhất của micro-batch (dòng có timestamp mới nhất)
    micro_batch_latest_df = (
        micro_batch_df
        .withColumn("row_num", F.row_number().over(Window.orderBy(F.col("DATE").desc())))
        .filter(F.col("row_num") == 1)  # Chỉ lấy dòng có timestamp mới nhất
        .drop("row_num")  # Loại bỏ cột row_num
    )
    # Kết hợp micro-batch với dữ liệu batch (historical_data_df)
    combined_df = (
        micro_batch_latest_df
        .unionByName(historical_data_df)  # Union với dữ liệu batch
    )
    
    # Tính toán các chỉ số SMA với các cửa sổ khác nhau
    window_spec = Window.orderBy("DATE").rowsBetween(Window.unboundedPreceding, 0)

    combined_df = combined_df.withColumn("SMA_5", F.avg("CLOSE").over(window_spec.rowsBetween(-4, 0)))
    combined_df = combined_df.withColumn("SMA_10", F.avg("CLOSE").over(window_spec.rowsBetween(-9, 0)))
    combined_df = combined_df.withColumn("SMA_20", F.avg("CLOSE").over(window_spec.rowsBetween(-19, 0)))
    combined_df = combined_df.withColumn("SMA_50", F.avg("CLOSE").over(window_spec.rowsBetween(-49, 0)))
    combined_df = combined_df.withColumn("SMA_100", F.avg("CLOSE").over(window_spec.rowsBetween(-99, 0)))
    combined_df = combined_df.withColumn("SMA_200", F.avg("CLOSE").over(window_spec.rowsBetween(-199, 0)))

    # Sắp xếp lại theo DATE
    combined_df = combined_df.orderBy("DATE", ascending=False)

    # Ghi ra kết quả
    combined_df.write.format("console").option("truncate", False).save()
    # combined_df.write \
    # .format("csv") \
    # .option("header", "true") \
    # .option("path", "gs://indicator-crypto/sma_results/BTC") \
    # .mode("overwrite") \
    # .save()


query = crypto_parsed_df.writeStream \
    .foreachBatch(process_batch) \
    .start()

query.awaitTermination()


24/12/13 22:40:44 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/k9/xss4jl9s24sg7_7zhcmrp2g00000gn/T/temporary-c0ebada1-cde1-4576-8139-dd3b88b955ad. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/12/13 22:40:44 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/12/13 22:40:45 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
24/12/13 22:40:46 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eve

+-------------------+--------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|DATE               |CLOSE   |SMA_5            |SMA_10           |SMA_20           |SMA_50           |SMA_100          |SMA_200          |
+-------------------+--------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|2024-12-13 00:00:00|100088.2|99074.06         |99215.75         |97511.875        |86722.98399999997|74511.1914       |68735.22729999998|
|2024-12-12 00:00:00|100050.1|99290.42         |98798.68000000001|97458.66650000002|86053.24219999996|74084.9978       |68577.15489999998|
|2024-12-11 00:00:00|101177.0|99260.38         |98379.0          |97378.11600000001|85399.84179999998|73675.8951       |68423.35274999999|
|2024-12-10 00:00:00|96655.1 |98999.55999999998|97907.84199999999|97038.9235       |84723.22479999997|73237.1151       |68260.20674999998|
|2024-12-09 00:00:00|97399.

24/12/13 22:41:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/13 22:41:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------------------------+--------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|DATE                      |CLOSE   |SMA_5            |SMA_10           |SMA_20           |SMA_50           |SMA_100          |SMA_200          |
+--------------------------+--------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|2024-12-13 22:41:04.301352|101312.0|99856.48000000001|99471.23999999999|97688.5365       |87386.08719999997|74944.6014       |68894.94369999999|
|2024-12-13 00:00:00       |100088.2|99074.06         |99215.75         |97511.875        |86722.98399999997|74511.1914       |68735.22729999998|
|2024-12-12 00:00:00       |100050.1|99290.42         |98798.68000000001|97458.66650000002|86053.24219999996|74084.9978       |68577.15489999998|
|2024-12-11 00:00:00       |101177.0|99260.38         |98379.0          |97378.11600000001|85399.84179999998|73675.8951     

24/12/13 22:42:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/13 22:42:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


...

+--------------------------+--------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|DATE                      |CLOSE   |SMA_5            |SMA_10           |SMA_20           |SMA_50           |SMA_100          |SMA_200          |
+--------------------------+--------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|2024-12-14 01:10:42.470924|101808.0|99917.6          |99501.8          |97703.8165       |87392.19919999997|74947.6574       |68896.4717       |
|2024-12-13 00:00:00       |99897.8 |99035.98         |99196.71         |97502.35500000001|86719.17599999996|74509.28739999999|68734.2753       |
|2024-12-12 00:00:00       |100050.1|99290.42         |98798.68000000001|97458.66650000002|86053.24219999996|74084.9978       |68577.15489999998|
|2024-12-11 00:00:00       |101177.0|99260.38         |98379.0          |97378.11600000001|85399.84179999998|73675.8951     

24/12/14 01:10:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/14 01:10:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------------------------+--------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|DATE                      |CLOSE   |SMA_5            |SMA_10           |SMA_20           |SMA_50           |SMA_100          |SMA_200          |
+--------------------------+--------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|2024-12-14 01:10:46.472453|101808.0|99917.6          |99501.8          |97703.8165       |87392.19919999997|74947.6574       |68896.4717       |
|2024-12-13 00:00:00       |99897.8 |99035.98         |99196.71         |97502.35500000001|86719.17599999996|74509.28739999999|68734.2753       |
|2024-12-12 00:00:00       |100050.1|99290.42         |98798.68000000001|97458.66650000002|86053.24219999996|74084.9978       |68577.15489999998|
|2024-12-11 00:00:00       |101177.0|99260.38         |98379.0          |97378.11600000001|85399.84179999998|73675.8951     

24/12/14 01:11:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/14 01:11:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------------------------+--------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|DATE                      |CLOSE   |SMA_5            |SMA_10           |SMA_20           |SMA_50           |SMA_100          |SMA_200          |
+--------------------------+--------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|2024-12-14 01:11:04.048824|101808.0|99917.6          |99501.8          |97703.8165       |87392.19919999997|74947.6574       |68896.4717       |
|2024-12-13 00:00:00       |99897.8 |99035.98         |99196.71         |97502.35500000001|86719.17599999996|74509.28739999999|68734.2753       |
|2024-12-12 00:00:00       |100050.1|99290.42         |98798.68000000001|97458.66650000002|86053.24219999996|74084.9978       |68577.15489999998|
|2024-12-11 00:00:00       |101177.0|99260.38         |98379.0          |97378.11600000001|85399.84179999998|73675.8951     